In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nfl-playing-surface-analytics/PlayList.csv
/kaggle/input/nfl-playing-surface-analytics/InjuryRecord.csv
/kaggle/input/nfl-playing-surface-analytics/PlayerTrackData.csv


First, let's see the InjuryRecord.csv.

In [2]:
inj=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/InjuryRecord.csv')

In [3]:
inj.head(3)

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
0,39873,39873-4,39873-4-32,Knee,Synthetic,1,1,1,1
1,46074,46074-7,46074-7-26,Knee,Natural,1,1,0,0
2,36557,36557-1,36557-1-70,Ankle,Synthetic,1,1,1,1


In [4]:
inj['DM_SUM']=inj['DM_M1']+inj['DM_M7']*6+inj['DM_M28']*21+inj['DM_M42']*14
inj.drop(columns=['DM_M1','DM_M7','DM_M28','DM_M42'])

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_SUM
0,39873,39873-4,39873-4-32,Knee,Synthetic,42
1,46074,46074-7,46074-7-26,Knee,Natural,7
2,36557,36557-1,36557-1-70,Ankle,Synthetic,42
3,46646,46646-3,46646-3-30,Ankle,Natural,1
4,43532,43532-5,43532-5-69,Ankle,Synthetic,42
...,...,...,...,...,...,...
100,44423,44423-13,NaN,Knee,Synthetic,1
101,31933,31933-20,NaN,Knee,Synthetic,1
102,47285,47285-4,NaN,Knee,Natural,7
103,37068,37068-19,NaN,Knee,Natural,7


Let's see an influence by surfacee.

In [5]:
inj.groupby(['Surface'])['DM_SUM'].describe()

,count,mean,std,min,25%,50%,75%,max
Surface,,,,,,,,
Natural,48.0,15.729167,17.021563,1.0,1.0,7.0,42.0,42.0
Synthetic,57.0,17.350877,17.276627,1.0,1.0,7.0,42.0,42.0


As for the number of days missed due to injury, synthetic turf is bigger than natural turf. 

In [6]:
inj.groupby(['Surface','BodyPart'])['DM_SUM'].describe()

count       mean        std   min   25%   50%   75%   max
Surface   BodyPart                                                           
Natural   Ankle      17.0  10.352941  15.353913   1.0   1.0   7.0   7.0  42.0
          Foot        5.0  39.200000   6.260990  28.0  42.0  42.0  42.0  42.0
          Heel        1.0   7.000000        NaN   7.0   7.0   7.0   7.0   7.0
          Knee       24.0  15.375000  16.534056   1.0   7.0   7.0  31.5  42.0
          Toes        1.0   7.000000        NaN   7.0   7.0   7.0   7.0   7.0
Synthetic Ankle      25.0  17.960000  18.219678   1.0   1.0   7.0  42.0  42.0
          Foot        2.0  35.000000   9.899495  28.0  31.5  35.0  38.5  42.0
          Knee       24.0  17.458333  17.505227   1.0   5.5   7.0  42.0  42.0
          Toes        6.0   8.500000   9.994999   1.0   2.5   7.0   7.0  28.0

In [7]:
wall=inj.groupby(['Surface','BodyPart'])['DM_SUM'].describe()
# Cut the parameters which number of samples is lower than 5. 
wall.loc[wall['count']>5]

count       mean        std  min  25%  50%   75%   max
Surface   BodyPart                                                        
Natural   Ankle      17.0  10.352941  15.353913  1.0  1.0  7.0   7.0  42.0
          Knee       24.0  15.375000  16.534056  1.0  7.0  7.0  31.5  42.0
Synthetic Ankle      25.0  17.960000  18.219678  1.0  1.0  7.0  42.0  42.0
          Knee       24.0  17.458333  17.505227  1.0  5.5  7.0  42.0  42.0
          Toes        6.0   8.500000   9.994999  1.0  2.5  7.0   7.0  28.0

As for the mean value of the table above, there are differences about Ankle and Knee by the Surface(Natural or Synthetic). 

Especially, players must take care of thier ankle when they play on synthetic turf.

Next, let's check PlayList.csv.

In [8]:
ply=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayList.csv')

In [9]:
ply.head(3)

,PlayerKey,GameID,PlayKey,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
0,26624,26624-1,26624-1-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,1,QB,QB
1,26624,26624-1,26624-1-2,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,2,QB,QB
2,26624,26624-1,26624-1-3,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,3,QB,QB


I would like to merge injury data and playlist data.

In [10]:
inj_ply=pd.merge(inj,ply)

In [11]:
inj_ply.groupby(['Surface','BodyPart','PlayType'])['DM_SUM'].describe()

count       mean        std   min  \
Surface   BodyPart PlayType                                                  
Natural   Ankle    Pass                    4.0  14.250000  18.714967   1.0   
                   Punt                    2.0   1.000000   0.000000   1.0   
                   Punt Returned           1.0   7.000000        NaN   7.0   
                   Rush                    7.0   9.428571  14.672618   1.0   
          Foot     Pass                    3.0  37.333333   8.082904  28.0   
                   Rush                    1.0  42.000000        NaN  42.0   
          Knee     Kickoff                 3.0  18.666667  20.207259   7.0   
                   Pass                    8.0   9.125000  13.611314   1.0   
                   Punt                    4.0  15.750000  17.500000   7.0   
                   Rush                    3.0  25.666667  17.616280   7.0   
Synthetic Ankle    Kickoff                 1.0  42.000000        NaN  42.0   
                   Pass                   11.0  20.000000  18.947295   1.0   
                   Punt                    1.0   7.000000        NaN   7.0   
                   Punt Returned           1.0   7.000000        NaN   7.0   
                   Rush                    7.0  17.428571  19.311975   1.0   
          Foot     Kickoff                 1.0  42.000000        NaN  42.0   
                   Punt Not Returned       1.0  28.000000        NaN  28.0   
          Knee     Kickoff                 2.0  21.500000  28.991378   1.0   
                   Kickoff Not Returned    1.0   7.000000        NaN   7.0   
                   Kickoff Returned        1.0   7.000000        NaN   7.0   
                   Pass                    6.0  16.333333  15.121728   7.0   
                   Punt                    2.0  42.000000   0.000000  42.0   
                   Punt Returned           1.0  42.000000        NaN  42.0   
                   Rush                    5.0  24.000000  19.248377   1.0   

                                           25%   50%    75%   max  
Surface   BodyPart PlayType                                        
Natural   Ankle    Pass                   5.50   7.0  15.75  42.0  
                   Punt                   1.00   1.0   1.00   1.0  
                   Punt Returned          7.00   7.0   7.00   7.0  
                   Rush                   1.00   7.0   7.00  42.0  
          Foot     Pass                  35.00  42.0  42.00  42.0  
                   Rush                  42.00  42.0  42.00  42.0  
          Knee     Kickoff                7.00   7.0  24.50  42.0  
                   Pass                   1.00   7.0   7.00  42.0  
                   Punt                   7.00   7.0  15.75  42.0  
                   Rush                  17.50  28.0  35.00  42.0  
Synthetic Ankle    Kickoff               42.00  42.0  42.00  42.0  
                   Pass                   4.00   7.0  42.00  42.0  
                   Punt                   7.00   7.0   7.00   7.0  
                   Punt Returned          7.00   7.0   7.00   7.0  
                   Rush                   1.00   7.0  35.00  42.0  
          Foot     Kickoff               42.00  42.0  42.00  42.0  
                   Punt Not Returned     28.00  28.0  28.00  28.0  
          Knee     Kickoff               11.25  21.5  31.75  42.0  
                   Kickoff Not Returned   7.00   7.0   7.00   7.0  
                   Kickoff Returned       7.00   7.0   7.00   7.0  
                   Pass                   7.00   7.0  22.75  42.0  
                   Punt                  42.00  42.0  42.00  42.0  
                   Punt Returned         42.00  42.0  42.00  42.0  
                   Rush                   7.00  28.0  42.00  42.0

Let's see the number of days player missed from the aspect of position and turf.

In [12]:
pos=inj_ply.groupby(['Surface','PositionGroup'])['DM_SUM'].describe()
pos.loc[pos['count']>5]

count       mean        std  min  25%   50%   75%  \
Surface   PositionGroup                                                      
Natural   DB               7.0  13.285714  15.606775  1.0  4.0   7.0  17.5   
          LB              11.0  14.909091  17.592354  1.0  4.0   7.0  24.5   
          WR               7.0   3.571429   3.207135  1.0  1.0   1.0   7.0   
Synthetic DB              12.0  21.666667  17.153496  1.0  7.0  17.5  42.0   
          LB              10.0  19.900000  18.260157  1.0  2.5  17.5  38.5   
          WR               9.0  23.555556  19.190565  1.0  7.0  28.0  42.0   

                          max  
Surface   PositionGroup        
Natural   DB             42.0  
          LB             42.0  
          WR              7.0  
Synthetic DB             42.0  
          LB             42.0  
          WR             42.0

The insight from the table above is that injury is occuring regardless of whether turf is Synthetic or Natural.
But, it looks like the number of days missed is influenced by the type of turf.

As for Ankle, number of days missed by Rush play on Synthetic turf is 17.428571, on Natural turf is 9.428571.
As for Knee, number of days missed by Pass play on Synthetic turf is 16.333333, on Natural turf is 9.125000.

And, regarding position, positions that player easily be injured are DB, LB, and WR.
for those 3 positions on synthetic turf, the number of days player missed is longer than natural turf.

I need to look into player's play for details.

In [13]:
trc=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayerTrackData.csv')

In [14]:
trc.head(2)

,PlayKey,time,event,x,y,dir,dis,o,s
0,26624-1-1,0.0,huddle_start_offense,87.46,28.93,288.24,0.01,262.33,0.13
1,26624-1-1,0.1,NaN,87.45,28.92,283.91,0.01,261.69,0.12
